In [2]:
import pandas as pd
import numpy as np

merge_df = pd.read_pickle("./pickled_merged_df.pkl")
business_info_df = pd.read_pickle("item_df.pkl")

In [3]:
business_info_df.info(memory_usage="deep")
business_info_df["business_id"] = business_info_df["business_id"].astype("string")
zero_cols = [ col for col, is_zero in ((business_info_df == 0).sum() == business_info_df.shape[0]).items() if is_zero ]
business_info_df.drop(zero_cols, axis=1, inplace=True)

for i, cat in zip(business_info_df.dtypes,business_info_df.columns):
    if i in [np.float64, np.float32, np.float16]:
        business_info_df[cat] = business_info_df[cat].astype("float32")
    elif i in [np.int64, np.int32, np.int16]:
        business_info_df[cat] = business_info_df[cat].astype("int32")
    else:
        business_info_df[cat] = business_info_df[cat].astype("string")
merge_df.drop(['review_id'], axis = 1, inplace = True)

business_info_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50763 entries, 0 to 50762
Columns: 730 entries, business_id to Elementary Schools
dtypes: float64(727), int64(2), object(1)
memory usage: 286.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50763 entries, 0 to 50762
Columns: 729 entries, business_id to Elementary Schools
dtypes: float32(726), int32(2), string(1)
memory usage: 144.8 MB


In [4]:
merge_df.head()

,user_id,business_id,stars,text,review_count,useful,funny,cool,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,Apparently Prides Osteria had a rough summer a...,63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
1,ak0TdVmGKo4pwqdJSTLwWw,bNZ3-0rse12NKdSVqQ30xw,4,"Came with friends, split the funghi pizza and ...",63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
2,ak0TdVmGKo4pwqdJSTLwWw,BVsIaKL-8QXVjt0Z9WoFWw,4,Went for late lunch had the combination seafoo...,63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
3,ak0TdVmGKo4pwqdJSTLwWw,4MClvr12OXBNvGu8h1yGpA,5,"We were super excited to try Sarma, having bee...",63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
4,ak0TdVmGKo4pwqdJSTLwWw,2vH58mhkEl8GdcDug1OwWg,5,So glad we made the trip to Woburn for Gene's ...,63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5


In [5]:
business_info_df.head()

,business_id,stars,review_count,is_open,Gastropubs,Food,Beer Gardens,Restaurants,Bars,American (Traditional),...,Video/Film Production,Oriental,Fischbroetchen,Striptease Dancers,Baby Gear & Furniture,Mailbox Centers,Printing Services,Notaries,Trampoline Parks,Elementary Schools
0,6iYb2HFDywm3zjuRg0shjw,4.0,86,1,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tCbdrRPZA0oiIYSmHG3J0w,4.0,126,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,D4JtQNTI4X3KcbzacDJsMw,3.5,169,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,jFYIsSb7r1QeESVUnXPHBw,4.5,11,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HPA_qyMEddpAEtFof02ixg,4.0,39,1,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import gensim

In [7]:
def get_docs(review_info, business_info, n = 10000):
    cols = business_info.columns
    for ix, review in review_info[:n].iterrows():
        business = business_info.loc[business_info['business_id'] == review["business_id"]]
        try:
            tag_ix = business.squeeze().to_numpy().nonzero()
            tags = cols[tag_ix] #all nonzero tags
        except:
            tags = [str(review["stars"])]
        yield review["text"], tags, review["stars"]
        
class MyCorpusSentsTagged:
    def __init__(self, merge_df, business_info, test_data = False, n = 20):
        self.test_data = test_data
        self.n = n
        self.merge_df = merge_df
        self.business_info = business_info

    def __iter__(self):
        for Doc, tags, stars in get_docs(self.merge_df, self.business_info, n = self.n):
            if self.test_data:
                yield Doc, stars
            else:
                yield gensim.models.doc2vec.TaggedDocument(Doc, tags)

def infer(model, text):
    return model.infer_vector(gensim.utils.simple_preprocess(text))

# Begin testing with Doc2Vec

In [274]:
corpus = MyCorpusSentsTagged(merge_df, business_info_df, n = 125000)
model = gensim.models.doc2vec.Doc2Vec(vector_size = 300,
                                      min_count=1,
                                      epochs = 3,
                                      workers = -1,
                                      max_vocab_size = None)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [319]:
x, y = [], []
corp = MyCorpusSentsTagged(merge_df, business_info_df, True, n = 50000)
for doc, stars in corp:
    x.append(infer(model, doc))
    y.append(stars)
y = np.array(y)

In [320]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
enc.fit(y.reshape(-1,1))

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [321]:
y = enc.transform(y.reshape(-1,1))

In [325]:
import numpy as np
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB

names = ["Decision Tree", "Random Forest", "Neural Net", "OLS", "Multinomial NB"]

classifiers = [    
    DecisionTreeClassifier(),                  
    RandomForestClassifier(n_estimators=75),   
    MLPClassifier(alpha=1e-5, hidden_layer_sizes=(50, 5)),
    linear_model.LinearRegression(),
    MultinomialNB()
]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)


# iterate over classifiers
for name, clf in zip(names, classifiers):
    try:
        clf.fit(np.array(x_train), y_train)
        score = clf.score(np.array(x_test), y_test)
    except:
        score = -1
    print(name, score)

min_score = 0
min_val = 0
for val in np.arange(2, 30, 4):
    clf = KNeighborsClassifier(val)
    clf.fit(x_train, y_train)
    tmp = clf.score(np.array(x_test), y_test)
    if tmp > min_score:
        min_val = val
        min_score = tmp
print("%s Nearest Neighbors" % min_val, min_score)

Decision Tree 0.2683
Random Forest 0.0035
Neural Net 0.0
OLS -0.008015432761377062
Multinomial NB -1
2 Nearest Neighbors 0.0813


In [339]:
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_test, classifiers[0].predict(x_test))

array([[[8839,  565],
        [ 559,   37]],

       [[8381,  854],
        [ 710,   55]],

       [[6262, 1662],
        [1647,  429]],

       [[4146, 2157],
        [2394, 1303]],

       [[5059, 2075],
        [2007,  859]]])

In [342]:
confusion_matrix(enc.inverse_transform(y_test), np.argmax(classifiers[0].predict(x_test), axis = 1))

array([[   0,    0,    0,    0,    0,    0],
       [  38,   59,  117,  209,  173,    0],
       [  45,   55,  171,  254,  240,    0],
       [ 127,  201,  429,  726,  593,    0],
       [ 236,  325,  764, 1303, 1069,    0],
       [ 160,  269,  610,  968,  859,    0]])

# Begin tfidf testing

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
vectorizer = TfidfVectorizer()


x, y = [], []
corp = MyCorpusSentsTagged(merge_df, business_info_df, True, n = 20000)
for doc, stars in corp:
    x.append(doc)
    y.append(stars)
y = np.array(y)

x = vectorizer.fit_transform(x)

enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1,1))

In [17]:
import numpy as np
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
names = ["Decision Tree", "Random Forest", "Neural Net", "OLS", "Multinomial NB"]

classifiers = [    
    DecisionTreeClassifier(),                  
    RandomForestClassifier(n_estimators=75),   
    MLPClassifier(alpha=1e-5, hidden_layer_sizes=(50, 5)),
]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)


# iterate over classifiers
for name, clf in zip(names, classifiers):
    try:
        clf.fit(np.array(x_train.todense()), y_train)
        score = clf.score(np.array(x_test.todense()), y_test)
    except Exception as E:
        print(E)
        score = -1
    print(name, score)

min_score = 0
min_val = 0
for val in np.arange(2, 30, 4):
    clf = KNeighborsClassifier(val)
    clf.fit(x_train, y_train)
    tmp = clf.score(np.array(x_test), y_test)
    if tmp > min_score:
        min_val = val
        min_score = tmp
print("%s Nearest Neighbors" % min_val, min_score)

Decision Tree 0.37325
Random Forest 0.12675
Neural Net 0.31425


ValueError: setting an array element with a sequence.